# Cyber Security: A Worst-Case Scenario

:::{admonition} Disclaimer
:class: danger
This chapter displays executable code (and the results of running said code) designed to perpetrate
serious cyber attacks against the provided example of a vulnerable application.
These code segments were strictly run against software running on servers owned and operated by the author of
this book.

Do **NOT, UNDER ANY CIRCUMSTANCES** attempt any of these attacks against a third-party service,
with the express consent of the owner of said service to test the attack.

By continuing to read this chapter, you acknowledge that we, the authors of this book, are in no way responsible
for the actions you choose to take with the knowledge you learn in this chapter.
You have been warned.
:::

When talking about software development, it's easy to forget about cyber security.
It's easy to assume that a security breach will never happen to you, especially if the code you're
writing is relatively simple or seemingly inconsequential.
It's assumptions like these which cause **catastrophic data breaches** every single day.

In this chapter, we aim to demonstrate the importance of security-minded development,
using as simple an example of a vulnerable application as we could cook up.

## A Desceptively Terrifying Example

Let's have a look at an example of a simple HTTP API server, written in Node.JS:

:::{danger}
The following code is strictly intended as an example of how **NOT** to write code.

It's buggy, chalk-full of security vulnerabilities, and it should **NOT UNDER ANY CIRCUMSTANCES**
be referred to as a reference application.
:::

```js
const express = require('express');
const { exec } = require('child_process');
const { Client } = require('pg');
const cookieParser = require('cookie-parser');
const app = express();
app.use(cookieParser());
const db = new Client();
db.connect();

// Use a Map to associate users with login tokens:
let tokens = new Map();

// Define a login route:
app.get('/login/:user/:password', (req, res) => {
	console.log(`Username: '${req.params.user}', Password: '${req.params.password}'`);
	// Compute the password hash:
	exec('echo -n "' + req.params.password + '" | md5sum', (err, hash) => {
		hash = hash.split(' ')[0];
		// Check the database for a user with the same username and password hash:
		db.query("SELECT * FROM users WHERE username='" + req.params.user + "' AND pwdhash='" + hash + "'", (err, users) => {
			// If the user wasn't found, don't let them sign in:
			if(users && users.rowCount === 0) {
				res.status(401).send('Authentication Failed');
			} else {
				// Generate a random auth token for the user:
				const token = Math.floor(Math.random() * 100).toString();
				tokens.set(req.params.user, token);

				// Send the token back as a cookie:
				res.cookie('user', req.params.user);
				res.cookie('token', token);
				res.status(200).send('Authentication succeeded!');
			}
		});
	});
});

// Define a password-protected route:
app.get('/secret', (req, res) => {
	console.log('Getting Secret');
	if(!req.cookies.token || req.cookies.token !== tokens.get(req.cookies.user)) {
		// Unauthorized users get rejected:
		res.status(401).send('Unauthorized request');
		console.log('Secret Not Sent!')
	} else {
		// Only authorized users can know the secret word:
		res.status(200).send('The secret word is "xylophone"');
		console.log('Secret Sent!')
	}
});

app.listen(8081, () => console.log('App is listening on port 8081'));
```

Despite being only 52 lines long, this simple application with only 2 API routes (one for authenticating users,
and one for accessing some secret information) is still teeming with security holes.
That said, it is still a completely functional application.

As a demonstration, let's consider a python application which first authenticates as a valid user (username:
*me*, password: *password123*), and then proceeds to access the application's secrets:

In [1]:
# Import dependencies:
from aiohttp import ClientSession
from urllib import parse

# Establish some basic info on the target:
host = 'localhost:8081'
username = 'me'
password = 'password123'


async with ClientSession() as session:
	# Send a login request to obtain an authorization token:
	url = 'http://{host}/login/{username}/{password}'.format(host=host, username=username, password=password)
	print('Login URL: {url}'.format(url=url))
	res = await session.get(url)
	print(await res.text() + '\n')

	# Print out the session's cookie details:
	if len(session.cookie_jar) > 0:
		for cookie in session.cookie_jar:
			print("Cookie '{key}' has value \"{value}\"".format(key=cookie.key, value=cookie.value))
	else:
		print('ERROR: Failed to extract cookies from response')

	# Request the secret data from the server:
	url = 'http://{host}/secret'.format(host=host)
	res = await session.get(url)
	print('\nSecret: ' + await res.text())

Login URL: http://localhost:8081/login/me/password123
Authentication succeeded!

Cookie 'user' has value "me"
Cookie 'token' has value "80"

Secret: The secret word is "xylophone"


As can be seen above, when presented with a valid username/password pair, the application functions
as expected, and spills it's secrets to a valid user.
Similarly, when provided with incorrect credentials, the application *should* reject that user's access.

In [2]:
# Establish some basic info on the target:
host = 'localhost:8081'
username = 'me'
password = 'wrongPassword'


async with ClientSession() as session:
	# Send a login request to obtain an authorization token:
	url = 'http://{host}/login/{username}/{password}'.format(host=host, username=username, password=password)
	print('Login URL: {url}'.format(url=url))
	res = await session.get(url)
	print(await res.text() + '\n')

	# Print out the session's cookie details:
	if len(session.cookie_jar) > 0:
		for cookie in session.cookie_jar:
			print("Cookie '{key}' has value \"{value}\"".format(key=cookie.key, value=cookie.value))
	else:
		print('ERROR: Failed to extract cookies from response')

	# Request the secret data from the server:
	url = 'http://{host}/secret'.format(host=host)
	res = await session.get(url)
	print('\nSecret: ' + await res.text())

Login URL: http://localhost:8081/login/me/wrongPassword
Authentication Failed

ERROR: Failed to extract cookies from response

Secret: Unauthorized request


However, while this might seem like the application is behaving completely as intended, it is important
to realize that it this isn't sufficient to determine that the application is secure or safe to use.

## Let the Cyber Attacks Begin!

In this chapter, we will use the above server code to explore it's various issues, and learn how those issues can be explioted
by attackers to achieve some seriously devistating results.

:::{admonition} Challenge Activity
:class: attention
Before you read on, read through the above server code and try to come up with a list of all of the different
security problems you are able to find in the code.
Write that list down somewhere, so you can reflect on it once you've completed this chapter.

```{hint}
:class: dropdown
For reference, our initial list consisted of 22 distinct security holes, but we're sure there are more
to be found!
```
:::